In [1]:
import pathlib
import pickle

import numpy as np
import scipy.sparse
import scipy.io
import pandas as pd

In [2]:
save_prefix = 'E:/HMCDA/data/preprocessed/HMCDA_processed/'

In [3]:
# load相互关系数据
mirna_disease = pd.read_csv('E:/HMCDA/data/raw/HMCDA/mirna_disease.dat', encoding='utf-8', delimiter='\t', names=['mirnaID', 'diseaseID'])
circrna_disease = pd.read_csv('E:/HMCDA/data/raw/HMCDA/circrna_disease.dat', encoding='utf-8', delimiter='\t', names=['circrnaID', 'diseaseID'])
circrna_mirna = pd.read_csv('E:/HMCDA/data/raw/HMCDA/circrna_mirna.dat', encoding='utf-8', delimiter='\t', names=['circrnaID', 'mirnaID'])
disease_idisease = pd.read_csv('E:/HMCDA/data/raw/HMCDA/disease_disease.dat', encoding='utf-8', delimiter='\t', names=['diseaseID', 'idiseaseID'])
# 疾病相互关系
num_mirna = 840
num_circrna = 1556
num_disease = 243

In [4]:
# circRNA-disease 训练集，验证集，测试集的关系index
train_val_test_idx = np.load('E:/HMCDA/data/raw/HMCDA/example5.npz')
train_idx = train_val_test_idx['train_idx']
val_idx = train_val_test_idx['val_idx']
test_idx = train_val_test_idx['test_idx']

circrna_disease = circrna_disease.loc[train_idx].reset_index(drop=True)

SyntaxError: EOL while scanning string literal (Temp/ipykernel_119312/904102721.py, line 2)

In [ ]:
dim = num_circrna + num_mirna + num_disease
# 0代表circRNA,1代表disease,2代表miRNA
type_mask = np.zeros((dim), dtype=int)
type_mask[num_circrna:num_circrna+num_disease] = 1
type_mask[num_circrna+num_disease:] = 2

#创建各个相互关系的邻接矩阵
adjM = np.zeros((dim, dim), dtype=int)
for _, row in circrna_disease.iterrows():
    cid = row['circrnaID'] - 1
    did = num_circrna + row['diseaseID'] - 1
    adjM[cid, did] += 1
    adjM[did, cid] += 1
for _, row in circrna_mirna.iterrows():
    cid = row['circrnaID'] - 1
    mid = num_circrna + num_disease + row['mirnaID'] - 1
    adjM[cid, mid] += 1
    adjM[mid, cid] += 1
for _, row in mirna_disease.iterrows():
    mid = num_circrna + num_disease + row['mirnaID'] - 1
    did = num_circrna + row['diseaseID'] - 1
    adjM[mid, did] += 1
    adjM[did, mid] += 1 
for _, row in disease_idisease.iterrows():
    did = num_circrna + row['diseaseID'] - 1
    idid = num_circrna + row['idiseaseID'] - 1
    adjM[did, idid] += 1 

In [ ]:

circrna_disease_list = {i: adjM[i, num_circrna:num_circrna+num_disease].nonzero()[0] for i in range(num_circrna)}
disease_circrna_list = {i: adjM[num_circrna + i, :num_circrna].nonzero()[0] for i in range(num_disease)}
disease_mirna_list = {i: adjM[num_circrna + i, num_circrna+num_disease:].nonzero()[0] for i in range(num_disease)}
mirna_disease_list = {i: adjM[num_circrna + num_disease + i, num_circrna:num_circrna+num_disease].nonzero()[0] for i in range(num_mirna)}
circrna_mirna_list = {i: adjM[i, num_circrna+num_disease:].nonzero()[0] for i in range(num_circrna)}
mirna_circrna_list = {i: adjM[num_circrna + num_disease + i, :num_circrna].nonzero()[0] for i in range(num_mirna)}
disease_disease_list = {i: adjM[num_circrna + i, num_circrna:num_circrna+num_disease].nonzero()[0] for i in range(num_disease)}

In [ ]:
# 构建元路径
# 0-1-0
c_d_c = []
for d, c_list in disease_circrna_list.items():
    c_d_c.extend([(c1, d, c2) for c1 in c_list for c2 in c_list])
c_d_c = np.array(c_d_c)
c_d_c[:, 1] += num_circrna
sorted_index = sorted(list(range(len(c_d_c))), key=lambda i : c_d_c[i, [0, 2, 1]].tolist())
c_d_c = c_d_c[sorted_index]

#0-2-0
c_m_c = []
for m, c_list in mirna_circrna_list.items():
    c_m_c.extend([(c1, m, c2) for c1 in c_list for c2 in c_list])
c_m_c = np.array(c_m_c)
c_m_c[:, 1] += (num_circrna + num_disease)
sorted_index = sorted(list(range(len(c_m_c))), key=lambda i : c_m_c[i, [0, 2, 1]].tolist())
c_m_c = c_m_c[sorted_index]

#2-1-2
m_d_m = []
for d, m_list in disease_mirna_list.items():
    m_d_m.extend([(m1, d, m2) for m1 in m_list for m2 in m_list])
m_d_m = np.array(m_d_m)
m_d_m += num_circrna
m_d_m[:, [0, 2]] += num_disease
sorted_index = sorted(list(range(len(m_d_m))), key=lambda i : m_d_m[i, [0, 2, 1]].tolist())
m_d_m = m_d_m[sorted_index]

#0-2-1-2-0
c_m_d_m_c = []
for m1, d, m2 in m_d_m:
    if len(mirna_circrna_list[m1 - num_circrna - num_disease]) == 0 or len(mirna_circrna_list[m2 - num_circrna - num_disease]) == 0:
        continue
    candidate_c1_list = np.random.choice(len(mirna_circrna_list[m1 - num_circrna - num_disease]), int(0.2 * len(mirna_circrna_list[m1 - num_circrna - num_disease])), replace=False)
    candidate_c1_list = mirna_circrna_list[m1 - num_circrna - num_disease][candidate_c1_list]
    candidate_c2_list = np.random.choice(len(mirna_circrna_list[m2 - num_circrna - num_disease]), int(0.2 * len(mirna_circrna_list[m2 - num_circrna - num_disease])), replace=False)
    candidate_c2_list = mirna_circrna_list[m2 - num_circrna - num_disease][candidate_c2_list]    
    c_m_d_m_c.extend([(c1, m1, d, m2, c2) for c1 in candidate_c1_list for c2 in candidate_c2_list])
c_m_d_m_c = np.array(c_m_d_m_c)
sorted_index = sorted(list(range(len(c_m_d_m_c))), key=lambda i : c_m_d_m_c[i, [0, 4, 1, 2, 3]].tolist())
c_m_d_m_c = c_m_d_m_c[sorted_index]

#1_0_1
d_c_d = []
for c, d_list in circrna_disease_list.items():
    d_c_d.extend([(d1, c, d2) for d1 in d_list for d2 in d_list])
d_c_d = np.array(d_c_d)
d_c_d[:, [0, 2]] += num_circrna
sorted_index = sorted(list(range(len(d_c_d))), key=lambda i : d_c_d[i, [0, 2, 1]].tolist())
d_c_d = d_c_d[sorted_index]

#1_2_1
d_m_d = []
for m, d_list in mirna_disease_list.items():
    d_m_d.extend([(d1, m, d2) for d1 in d_list for d2 in d_list])
d_m_d = np.array(d_m_d)
d_m_d[:, 1] += (num_circrna + num_disease)
d_m_d[:, [0, 2]] += num_circrna
sorted_index = sorted(list(range(len(d_m_d))), key=lambda i : d_m_d[i, [0, 2, 1]].tolist())
d_m_d = d_m_d[sorted_index]
#1-1
d_d = disease_idisease.to_numpy(dtype=np.int32) - 1
sorted_index = sorted(list(range(len(d_d))), key=lambda i : d_d[i].tolist())
d_d = d_d[sorted_index]

In [ ]:
expected_metapaths = [
    [(0, 1, 0), (0, 2, 1, 2, 0), (0, 2, 0)],
    [(1, 0, 1), (1, 2, 1), (1, 1)]
]
# create the directories if they do not exist
for i in range(len(expected_metapaths)):
    pathlib.Path(save_prefix + '{}'.format(i)).mkdir(parents=True, exist_ok=True)

metapath_indices_mapping = {(0, 1, 0): c_d_c,
                            (0, 2, 1, 2, 0): c_m_d_m_c,
                            (0, 2, 0): c_m_c,
                            (1, 0, 1): d_c_d,
                            (1, 2, 1): d_m_d,
                            (1, 1): d_d}
# write all things 
target_idx_lists = [np.arange(num_circrna), np.arange(num_disease)]
offset_list = [0, num_circrna]
for i, metapaths in enumerate(expected_metapaths):
    for metapath in metapaths:
        edge_metapath_idx_array = metapath_indices_mapping[metapath]
        
        with open(save_prefix + '{}/'.format(i) + '-'.join(map(str, metapath)) + '_idx.pickle', 'wb') as out_file:
            target_metapaths_mapping = {}
            left = 0
            right = 0
            for target_idx in target_idx_lists[i]:
                while right < len(edge_metapath_idx_array) and edge_metapath_idx_array[right, 0] == target_idx + offset_list[i]:
                    right += 1
                target_metapaths_mapping[target_idx] = edge_metapath_idx_array[left:right, ::-1]
                left = right
            pickle.dump(target_metapaths_mapping, out_file)

        #np.save(save_prefix + '{}/'.format(i) + '-'.join(map(str, metapath)) + '_idx.npy', edge_metapath_idx_array)
        
        with open(save_prefix + '{}/'.format(i) + '-'.join(map(str, metapath)) + '.adjlist', 'w') as out_file:
            left = 0
            right = 0
            for target_idx in target_idx_lists[i]:
                while right < len(edge_metapath_idx_array) and edge_metapath_idx_array[right, 0] == target_idx + offset_list[i]:
                    right += 1
                neighbors = edge_metapath_idx_array[left:right, -1] - offset_list[i]
                neighbors = list(map(str, neighbors))
                if len(neighbors) > 0:
                    out_file.write('{} '.format(target_idx) + ' '.join(neighbors) + '\n')
                else:
                    out_file.write('{}\n'.format(target_idx))
                left = right

scipy.sparse.save_npz(save_prefix + 'adjM.npz', scipy.sparse.csr_matrix(adjM))
np.save(save_prefix + 'node_types.npy', type_mask)

In [ ]:
circrna_disease = pd.read_csv('E:/HMCDA/data/raw/HMCDA/circrna_disease.dat', encoding='utf-8', delimiter='\t', names=['circrnaID', 'diseaseID'])
circrna_disease = circrna_disease[['circrnaID', 'diseaseID']].to_numpy()
circrna_disease = circrna_disease - 1
np.save(save_prefix + 'circrna_disease.npy', circrna_disease)

In [ ]:
# output positive and negative samples for training, validation and testing

np.random.seed(453289)
save_prefix = 'E:/HMCDA/data/preprocessed/HMCDA_processed/'
num_circrna = 1556
num_disease = 243
circrna_disease = np.load('E:/HMCDA/data/preprocessed/HMCDA_processed/circrna_disease.npy')
train_val_test_idx = np.load('E:/HMCDA/data/raw/HMCDA/example5.npz')
train_idx = train_val_test_idx['train_idx']
val_idx = train_val_test_idx['val_idx']
test_idx = train_val_test_idx['test_idx']

neg_candidates = []
counter = 0
for i in range(num_circrna):
    for j in range(num_disease):
        if counter < len(circrna_disease):
            if i == circrna_disease[counter, 0] and j == circrna_disease[counter, 1]:
                counter += 1
            else:
                neg_candidates.append([i, j])
        else:
            neg_candidates.append([i, j])
neg_candidates = np.array(neg_candidates)

idx = np.random.choice(len(neg_candidates), len(val_idx) + len(test_idx), replace=False)
val_neg_candidates = neg_candidates[sorted(idx[:len(val_idx)])]
test_neg_candidates = neg_candidates[sorted(idx[len(val_idx):])]

train_circrna_disease = circrna_disease[train_idx]
train_neg_candidates = []
counter = 0
for i in range(num_circrna):
    for j in range(num_disease):
        if counter < len(train_circrna_disease):
            if i == train_circrna_disease[counter, 0] and j == train_circrna_disease[counter, 1]:
                counter += 1
            else:
                train_neg_candidates.append([i, j])
        else:
            train_neg_candidates.append([i, j])
train_neg_candidates = np.array(train_neg_candidates)

np.savez(save_prefix + 'train_val_test_neg_circrna_disease.npz',
         train_neg_circrna_disease=train_neg_candidates,
         val_neg_circrna_disease=val_neg_candidates,
         test_neg_circrna_disease=test_neg_candidates)
np.savez(save_prefix + 'train_val_test_pos_circrna_disease.npz',
         train_pos_circrna_disease=circrna_disease[train_idx],
         val_pos_circrna_disease=circrna_disease[val_idx],
         test_pos_circrna_disease=circrna_disease[test_idx])